In [8]:
# import os

# # Get the current working directory
# current_dir = os.getcwd()

# # Go one step back
# parent_dir = os.path.dirname(current_dir)

# # Change the working directory to the parent directory
# os.chdir(parent_dir)

# # Print the new working directory to confirm
# print("New working directory:", os.getcwd())

In [13]:
ct_dir = os.path.join(original_meshes_dir, 'CT')
print("Contents of CT directory:")
print(os.listdir(ct_dir))


Contents of CT directory:
['case_1.stl', 'case_101.stl', 'case_102.stl', 'case_104.stl', 'case_109.stl', 'case_11.stl', 'case_112.stl', 'case_113.stl', 'case_114.stl', 'case_12.stl', 'case_128.stl', 'case_13.stl', 'case_135.stl', 'case_138.stl', 'case_139.stl', 'case_14.stl', 'case_140.stl', 'case_141.stl', 'case_142.stl', 'case_143.stl', 'case_144.stl', 'case_147.stl', 'case_149.stl', 'case_150.stl', 'case_153.stl', 'case_154.stl', 'case_155.stl', 'case_157.stl', 'case_159.stl', 'case_16.stl', 'case_161.stl', 'case_163.stl', 'case_165.stl', 'case_166.stl', 'case_167.stl', 'case_168.stl', 'case_169.stl', 'case_170.stl', 'case_171.stl', 'case_172.stl', 'case_173.stl', 'case_174.stl', 'case_175.stl', 'case_176.stl', 'case_177.stl', 'case_178.stl', 'case_179.stl', 'case_18.stl', 'case_180.stl', 'case_181.stl', 'case_182.stl', 'case_183.stl', 'case_184.stl', 'case_185.stl', 'case_186.stl', 'case_188.stl', 'case_190.stl', 'case_191.stl', 'case_192.stl', 'case_193.stl', 'case_194.stl', 'case

In [20]:
print(os.listdir(base_dir_2))


['case_1.vtk', 'case_101.vtk', 'case_101_0.vtk', 'case_101_1.vtk', 'case_101_10.vtk', 'case_101_2.vtk', 'case_101_20.vtk', 'case_101_3.vtk', 'case_101_4.vtk', 'case_101_5.vtk', 'case_101_6.vtk', 'case_101_7.vtk', 'case_101_8.vtk', 'case_101_9.vtk', 'case_102.vtk', 'case_102_0.vtk', 'case_102_1.vtk', 'case_102_10.vtk', 'case_102_2.vtk', 'case_102_20.vtk', 'case_102_3.vtk', 'case_102_4.vtk', 'case_102_5.vtk', 'case_102_6.vtk', 'case_102_7.vtk', 'case_102_8.vtk', 'case_102_9.vtk', 'case_104.vtk', 'case_104_0.vtk', 'case_104_1.vtk', 'case_104_10.vtk', 'case_104_2.vtk', 'case_104_20.vtk', 'case_104_3.vtk', 'case_104_4.vtk', 'case_104_5.vtk', 'case_104_6.vtk', 'case_104_7.vtk', 'case_104_8.vtk', 'case_104_9.vtk', 'case_109.vtk', 'case_109_0.vtk', 'case_109_1.vtk', 'case_109_10.vtk', 'case_109_2.vtk', 'case_109_20.vtk', 'case_109_3.vtk', 'case_109_4.vtk', 'case_109_5.vtk', 'case_109_6.vtk', 'case_109_7.vtk', 'case_109_8.vtk', 'case_109_9.vtk', 'case_11.vtk', 'case_112.vtk', 'case_112_0.vtk', 

In [23]:
import pyvista as pv
import numpy as np
import trimesh
import os
import vtk
import pandas as pd

def readvtk(filename, dataarrays=True):
    """Read image in vtk format."""
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(filename)
    reader.Update()
    if not dataarrays:
        for i in range(reader.GetNumberOfPointArrays()):
            arrayname = reader.GetPointArrayName(i)
            reader.SetPointArrayStatus(arrayname, 0)
        for i in range(reader.GetNumberOfCellArrays()):
            arrayname = reader.GetCellArrayName(i)
            reader.SetPointArrayStatus(arrayname, 0)
        reader.Update()
    return reader.GetOutput()

def addpoint_show(p, radius=0.6, color=[0.0, 0.0, 1.0]):
    """Add a point to the visualizer."""
    point = vtk.vtkSphereSource()
    point.SetCenter(p)
    point.SetRadius(radius)
    point.SetPhiResolution(100)
    point.SetThetaResolution(100)

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(point.GetOutputPort())

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)

    return actor

def align_mesh(lm1, lm2, mesh1, mesh2, plot=False, weights=None, reflection=False, translation=True, scale=False, return_cost=True):
    """Align a set of two landmarks."""
    transform, lmA, _ = trimesh.registration.procrustes(lm2, lm1, weights=weights, reflection=reflection, translation=translation, scale=scale, return_cost=return_cost)
    aligned = mesh2.transform(transform, inplace=False, transform_all_input_vectors=True)

    if plot:    
        plotter = pv.Plotter(notebook=0)
        plotter.background_color = 'w'
        plotter.add_mesh(pv.PolyData(lm1[0]), color='r')
        plotter.add_mesh(pv.PolyData(lm1[1]), color='g')
        plotter.add_mesh(pv.PolyData(lm1[2]), color='b')
        plotter.add_mesh(pv.PolyData(lm1[3]), color='y')
        plotter.add_mesh(pv.PolyData(lm1[4]), color='pink')
        plotter.add_mesh(mesh1, color='gray', opacity=0.1)

        plotter.add_mesh(pv.PolyData(lmA[0]), color='r')
        plotter.add_mesh(pv.PolyData(lmA[1]), color='g')
        plotter.add_mesh(pv.PolyData(lmA[2]), color='b')
        plotter.add_mesh(pv.PolyData(lmA[3]), color='y')
        plotter.add_mesh(pv.PolyData(lmA[4]), color='pink')
        plotter.add_mesh(aligned, color='red', opacity=0.1)
        
        plotter.show()

    return aligned

def readstl(filename):
    """Read STL file using vtk."""
    reader = vtk.vtkSTLReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def estimate_center(LAA_edge):
    """Estimate the center of the base."""
    npoints = LAA_edge.GetNumberOfPoints()
    cent = []

    for i in range(npoints):
        points = LAA_edge.GetPoint(i)
        if i == 0:
            cent.append(points)
        else:
            cent.append(np.add(cent[-1], points))

    center = np.array([cent[-1][0] / npoints, cent[-1][1] / npoints, cent[-1][2] / npoints])

    return center

# Read the Excel file
df1 = pd.read_excel('ostium_prediction_marta_final_endp.xlsx', engine='openpyxl')

# Initialize cases list
list_case = []
for case in list(df1.Patient_ID):
    case_number = int(case.split('_')[1])
    if case_number != 208:  # Exclude case 208
        list_case.append(case_number)

list_centers = []
list_mesh = []
list_case_b = []

# Loop through each case
for case, ind in zip(list_case, range(len(list_case))):
    try:
        # Read left seed points
        list_coords_left = []
        with open(f'seeds_left/Bordeaux_Case{case}.txt') as f:
            for line in f:
                l = line.strip()
                coord = np.array((float(l.split(',')[0]), float(l.split(',')[1]), float(l.split(',')[2])))
                list_coords_left.append(coord)

        lspv = list_coords_left[0]
        lipv = list_coords_left[-1]

        # Read right seed points
        list_coords_right = []
        with open(f'seeds_right/Bordeaux_Case{case}.txt') as f:
            for line in f:
                l = line.strip()
                coord = np.array((float(l.split(',')[0]), float(l.split(',')[1]), float(l.split(',')[2])))
                list_coords_right.append(coord)

        rspv = list_coords_right[0]
        ripv = list_coords_right[-1]

    except FileNotFoundError:
        print(f'No points found for patient {case}. Continuing with the next case...')
        continue

    # Updated path to read the mesh from CT directory
    base_dir = 'Meshes/Original_meshes/CT'
    file_name = f'case_{case}.stl'
    file_path = os.path.join(base_dir, file_name)

    print(f'Reading mesh from: {file_path}')  # For debugging

    # Now read the mesh using the relative path
    mesh = pv.read(file_path)

    endpoint = np.array((df1.endp_x[ind], df1.endp_y[ind], df1.endp_z[ind]))
    ostium = np.array((df1.ost_x[ind], df1.ost_y[ind], df1.ost_z[ind]))

    # Updated path to read the second mesh from the VTK_Final_meshes_centered directory
    base_dir_2 = 'Meshes\VTK_Final_meshes_centered'
    file_name_2 = f'case_{case}.vtk'
    file_path_2 = os.path.join(base_dir_2, file_name_2)

    print(f'Reading second mesh from: {file_path_2}')  # For debugging

    # Check if the file exists in the second directory
    if not os.path.isfile(file_path_2):
        print(f'File not found for case {case} in the second directory: {file_path_2}. Skipping this case.')
        continue

    try:
        mesh2 = pv.read(file_path_2)
    except Exception as e:
        print(f'Error reading second mesh for case {case}: {e}')
        continue
    
    centro_masa = estimate_center(mesh2)
    # Choose the anatomical landmarks (ostium / endpoint / center of mass / pulmonary veins)
    list_centers.append([ostium, endpoint, lspv, rspv, lipv, ripv])
    list_mesh.append(mesh)
    list_case_b.append(case)


Reading mesh from: Meshes/Original_meshes/CT\case_54.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_54.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_18.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_18.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_47.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_47.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_14.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_14.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_36.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_36.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_50.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_50.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_59.stl
Reading second mesh from: Meshes\VTK_Final_meshes_centered\case_59.vtk
Reading mesh from: Meshes/Original_meshes/CT\case_25.stl
Reading second mesh from: Meshes\VTK_Final_mesh

In [24]:
list_mesh

[PolyData (0x1be0be9bc40)
   N Cells:    26680
   N Points:   13342
   N Strips:   0
   X Bounds:   -4.846e+01, 6.192e+01
   Y Bounds:   9.298e+01, 1.739e+02
   Z Bounds:   1.522e+03, 1.610e+03
   N Arrays:   0,
 PolyData (0x1be0be468e0)
   N Cells:    27366
   N Points:   13685
   N Strips:   0
   X Bounds:   -6.764e+01, 5.666e+01
   Y Bounds:   9.896e+01, 1.763e+02
   Z Bounds:   1.561e+03, 1.633e+03
   N Arrays:   0,
 PolyData (0x1be0bd8a880)
   N Cells:    42646
   N Points:   21325
   N Strips:   0
   X Bounds:   -5.253e+01, 5.613e+01
   Y Bounds:   -1.806e+02, -1.022e+02
   Z Bounds:   1.580e+03, 1.654e+03
   N Arrays:   0,
 PolyData (0x1be0bf6c820)
   N Cells:    33498
   N Points:   16751
   N Strips:   0
   X Bounds:   -6.670e+01, 5.439e+01
   Y Bounds:   1.335e+02, 2.272e+02
   Z Bounds:   1.253e+03, 1.347e+03
   N Arrays:   0,
 PolyData (0x1be0bf6ca60)
   N Cells:    30082
   N Points:   15043
   N Strips:   0
   X Bounds:   -5.457e+01, 6.143e+01
   Y Bounds:   1.106e+02, 1.

In [26]:
# ALIGNMENT

import os

# Create a new directory for saving aligned meshes
output_dir = 'Meshes_aligned'  # Folder name
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

for i, (center, c) in enumerate(zip(list_centers, list_case_b)):
    # Align the rest of the meshes with the first one
    aligned = align_mesh(list_centers[63], list_centers[i], list_mesh[63], list_mesh[i], plot=False)

    # Save the aligned mesh in the new folder with a relative path
    c = str(c)
    output_file_path = os.path.join(output_dir, f'case_{c}.stl')
    aligned.save(output_file_path)

In [27]:
list_centers

[[array([ -20.099598  ,  139.91498844, 1583.7985712 ]),
  array([ -29.93832435,  159.25835369, 1607.67605442]),
  array([ -27.51639269,  124.99620932, 1589.24954792]),
  array([  21.31242257,  139.55729745, 1586.57850426]),
  array([ -23.30480367,  112.09999265, 1566.49671168]),
  array([  37.73858407,  128.45353996, 1557.52420587])],
 [array([ -28.62518257,  144.51545802, 1616.50048828]),
  array([ -67.24236227,  146.54845667, 1609.46950402]),
  array([ -33.02446707,  129.0869455 , 1624.18126112]),
  array([  35.80648115,  143.95634071, 1597.13757665]),
  array([ -26.96824751,  109.55749289, 1604.92205328]),
  array([  27.50115879,  122.06265581, 1586.06240647])],
 [array([  28.75661085, -143.24613433, 1630.32892008]),
  array([  52.57099601, -180.17820527, 1634.28200003]),
  array([  22.97811039, -130.09259977, 1645.52111618]),
  array([ -38.69583448, -143.30361343, 1633.5395789 ]),
  array([  29.14288175, -111.19517502, 1625.18170776]),
  array([ -31.73814062, -123.80492999, 1610.08